In [1]:
require 'nn'
require 'optim'
require 'mnist'
require 'dataset-mnist'

In [2]:
torch.manualSeed(0)
torch.setnumthreads(4)

In [3]:
--classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20 }
classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 }
geometry = { 32, 32 }

net = nn.Sequential()

net:add(nn.SpatialConvolution(1, 6, 5, 5))
net:add(nn.ReLU())
net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

net:add(nn.SpatialConvolution(6, 16, 5, 5))
net:add(nn.ReLU())
net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

net:add(nn.View(16*5*5))
net:add(nn.Linear(16*5*5, 120))
net:add(nn.ReLU())
net:add(nn.Linear(120, 84))
net:add(nn.ReLU())
net:add(nn.Linear(84, #classes))
net:add(nn.LogSoftMax())

In [4]:
parameters, gradParameters = net:getParameters()

In [5]:
criterion = nn.ClassNLLCriterion()

In [6]:
n_training_patches = 60000--60000
n_testing_patches = 10000--10000

train_data = mnist.loadTrainSet(n_training_patches, geometry)
train_data_inverted = mnist.loadTrainSet(n_training_patches, geometry)

for i = 1, n_training_patches do
    train_data_inverted[i][1]:apply(function(x) 
        return 255 - x
    end)

--     original = train_data_inverted[i][2]
--     prepend = torch.zeros(10)
--     combined = torch.cat(prepend, original)
    --print(combined)
    
    --train_data_inverted.labels[i] = train_data_inverted.labels[i] + 10
    --torch.reshape(train_data_inverted[i][2], combined, 20)
--     train_data_inverted[i][2]:resize(20, 1)
--     train_data_inverted[i][2][12] = 15
--     train_data_inverted[i][2]:apply(function(x)
--         return 5 - x
--     end)
end

-- print(train_data_inverted[1][1])
-- print(train_data_inverted[1][2])

train_data:normalizeGlobal(mean, std)
train_data_inverted:normalizeGlobal(mean, std)

test_data = mnist.loadTestSet(n_testing_patches, geometry)
test_data_inverted = mnist.loadTestSet(n_testing_patches, geometry)

for i = 1, n_testing_patches do
    test_data_inverted[i][1]:apply(function(x)
            return 255 - x
    end)
    --test_data_inverted.data[i] = 255 - test_data_inverted.data[i]
end

test_data:normalizeGlobal(mean, std)
test_data_inverted:normalizeGlobal(mean, std)

<mnist> done	


<mnist> done	


 0
 0
 0
 0
 0
 1
 0
 0
 0
 0
[torch.DoubleTensor of size 10]



<mnist> done	


<mnist> done	


In [7]:
confusion = optim.ConfusionMatrix(classes)
epoch_limit = 5

In [8]:
actual_train = train_data_inverted
actual_test = test_data_inverted

In [9]:
batch_size = 10
for epoch = 1, epoch_limit do
    print("Epoch: " .. epoch)
    for t = 1, actual_train:size(), batch_size do
        local inputs = torch.Tensor(batch_size, 1, geometry[1], geometry[2])
        local targets = torch.Tensor(batch_size)
        local k = 1

        for i = t, math.min(t + batch_size - 1, actual_train:size()) do
            local sample = actual_train[i]
            local input = sample[1]:clone()
            
            local _, target = sample[2]:clone():max(1)

            target = target:squeeze()
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end

        local feval = function(x)
            collectgarbage()

            if x ~= parameters then
                parameters:copy(x)
            end
            gradParameters:zero()

            local outputs = net:forward(inputs)
            local f = criterion:forward(outputs, targets)
            local df_do = criterion:backward(outputs, targets)
            net:backward(inputs, df_do)

            for i = 1, batch_size do
                confusion:add(outputs[i], targets[i])
            end

            return f, gradParameters
        end

        sgd_state = sgd_state or {
            learning_rate = 0.06,
            momentum = 0,
            learning_rate_decay = 5e-7
        }
        optim.sgd(feval, parameters, sgd_state)

    end
    
    print(confusion)
    confusion:zero()
end



Epoch: 1	


ConfusionMatrix:
[[    4874       4      20      18       5      18      25     896      54       9]   82.289% 	[class: 1]
 [      23    4855      59      29       7       7      10    1666      50      36]   72.011% 	[class: 2]
 [     645      34    2965     404      87      14     174    1546      75      14]   49.765% 	[class: 3]
 [     548      41     213    3419       8     102      25    1572     122      81]   55.766% 	[class: 4]
 [     100      18      17      74    3105       0     192    2076      18     242]   53.150% 	[class: 5]
 [     680      41      97     775     102    1751     146    1321     424      84]   32.300% 	[class: 6]
 [     415      70      93     147      55      50    3416    1645      24       3]   57.722% 	[class: 7]
 [     129      76      55      31      51       7      15    5634      30     237]   89.928% 	[class: 8]
 [     476     184     174     429      46     204     123    1722    2419      74]   41.343% 	[class: 9]
 [     181      46      19   

ConfusionMatrix:
[[    5711       1      32       9       6      47      46       8      47      16]   96.421% 	[class: 1]
 [       1    6564      51      25       3      32       8      10      34      14]   97.360% 	[class: 2]
 [      39      40    5366     104      95      19      83      84     109      19]   90.064% 	[class: 3]
 [      31      31     153    5462       1     188      14      73     118      60]   89.088% 	[class: 4]
 [       5      20      51       1    5355       8      79      12      25     286]   91.664% 	[class: 5]
 [      63      31      31     181      50    4778      98      11     129      49]   88.139% 	[class: 6]
 [      34      29      49       0      58      92    5616       4      35       1]   94.897% 	[class: 7]
 [      18      40      76      21      51      17       0    5780      17     245]   92.259% 	[class: 8]
 [      24     111      50     163      38     157      63      20    5125     100]   87.592% 	[class: 9]
 [      42      27      22   

ConfusionMatrix:
[[    5775       1      20       6       6      24      37       6      33      15]   97.501% 	[class: 1]
 [       2    6619      45      11       6       8       4      10      29       8]   98.176% 	[class: 2]
 [      25      47    5613      60      37      12      30      62      64       8]   94.209% 	[class: 3]
 [      19      12      95    5716       1     113       9      47      72      47]   93.231% 	[class: 4]
 [       5      17      26       0    5533       3      47      12      19     180]   94.711% 	[class: 5]
 [      38      16      13      84      12    5125      49       4      45      35]   94.540% 	[class: 6]
 [      28      15      17       1      36      47    5747       0      27       0]   97.111% 	[class: 7]
 [      10      23      63      26      34       7       0    5973      18     111]   95.339% 	[class: 8]
 [      22      71      38      82      24      55      54      16    5422      67]   92.668% 	[class: 9]
 [      31      19       5   

  _mat_flat : LongTensor - size: 100
  valids : FloatTensor - size: 10
  mat : LongTensor - size: 10x10
  averageUnionValid : 0.9044608771801
  _targ_idx : LongTensor - empty
  averageValid : 0.94955144524574
  classes : 
    {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
      6 : 6
      7 : 7
      8 : 8
      9 : 9
      10 : 10
    }
  _prediction : FloatTensor - size: 10
  _pred_idx : LongTensor - size: 1
  nclasses : 10
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 10
  totalValid : 0.95
}
Epoch: 4	


ConfusionMatrix:
[[    5811       2      19       3       6      16      29       5      20      12]   98.109% 	[class: 1]
 [       1    6640      36      15       8       0       4       9      23       6]   98.487% 	[class: 2]
 [      20      41    5712      52      20       5      13      48      41       6]   95.871% 	[class: 3]
 [      12       7      71    5823       2      84       5      34      59      34]   94.976% 	[class: 4]
 [       4      14      15       1    5619       2      34       6      14     133]   96.183% 	[class: 5]
 [      23       8       8      49       4    5211      40       4      37      37]   96.126% 	[class: 6]
 [      24      12       8       0      23      37    5790       0      24       0]   97.837% 	[class: 7]
 [       7      18      45      18      21       7       0    6061      19      69]   96.744% 	[class: 8]
 [      22      47      32      63      17      34      38      13    5536      49]   94.616% 	[class: 9]
 [      26      14       2   

  _prediction : FloatTensor - size: 10
  _pred_idx : LongTensor - size: 1
  nclasses : 10
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 10
  totalValid : 0.9631
}
Epoch: 5	


ConfusionMatrix:
[[    5831       1      16       3       6      12      24       4      15      11]   98.447% 	[class: 1]
 [       1    6651      33      13       8       0       3       9      18       6]   98.650% 	[class: 2]
 [      14      31    5777      38      12       1       9      36      33       7]   96.962% 	[class: 3]
 [      10       7      56    5878       2      66       2      28      54      28]   95.873% 	[class: 4]
 [       3      13      13       1    5658       1      23       7      11     112]   96.850% 	[class: 5]
 [      17       4       7      39       3    5249      37       5      31      29]   96.827% 	[class: 6]
 [      23      10       6       1      16      33    5809       0      20       0]   98.158% 	[class: 7]
 [       3      15      36      12      21       4       0    6099      17      58]   97.350% 	[class: 8]
 [      19      41      32      49      19      29      29      12    5578      43]   95.334% 	[class: 9]
 [      21      10       2   

In [10]:
confusion:zero()

for t = 1, actual_test:size(), 10 do
    local inputs = torch.Tensor(batch_size, 1, geometry[1], geometry[2])
    local targets = torch.Tensor(batch_size)
    local k = 1
    
    for i = t, math.min(t + batch_size - 1, actual_test:size()) do
        local sample = actual_test[i]
        local input = sample[1]:clone()
        local _, target = sample[2]:clone():max(1)
        target = target:squeeze()
        inputs[k] = input
        targets[k] = target
        k = k + 1
    end
    
    local preds = net:forward(inputs)
    
    for i = 1, batch_size do
        confusion:add(preds[i], targets[i])
    end
end

print(confusion)

ConfusionMatrix:
[[     974       0       1       0       0       1       0       1       3       0]   99.388% 	[class: 1]
 [       0    1131       2       0       0       1       0       0       1       0]   99.648% 	[class: 2]
 [       5      11     998      10       1       0       0       3       4       0]   96.705% 	[class: 3]
 [       1       0       1     998       0       2       0       4       2       2]   98.812% 	[class: 4]
 [       0       1       3       0     961       0       1       0       2      14]   97.862% 	[class: 5]
 [       3       1       0      12       2     870       1       0       1       2]   97.534% 	[class: 6]
 [      15       4       1       1       8      38     885       0       6       0]   92.380% 	[class: 7]
 [       0       7      13       4       1       2       0     989       1      11]   96.206% 	[class: 8]
 [       6       4       3      30       3       6       2       4     901      15]   92.505% 	[class: 9]
 [       5       7       0   